In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

import joblib

In [7]:
X_train = np.load('../Data/X_train.npy')
X_test = np.load('../Data/X_test.npy')

X_train_scaled = np.load('../Data/X_train_scaled.npy')
X_test_scaled = np.load('../Data/X_test_scaled.npy')

y_train = np.load('../Data/y_train.npy')
y_test = np.load('../Data/y_test.npy')


print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (2262300, 78)
Test shape: (565576, 78)


In [25]:
print("Before balancing:")
print(pd.Series(y_train).value_counts())

Before balancing:
0     1817055
4      184099
10     127043
2      102420
3        8234
7        6348
11       4717
6        4637
5        4399
1        1565
12       1206
14        522
9          29
13         17
8           9
Name: count, dtype: int64


In [31]:
attack_total = np.sum(y_train != 0)

rus = RandomUnderSampler(
    sampling_strategy={0: attack_total * 3},   # 3:1 ratio (safe & realistic)
    random_state=42
)

X_under, y_under = rus.fit_resample(X_train_scaled, y_train)

print("After RUS:\n", pd.Series(y_under).value_counts())


MemoryError: Unable to allocate 530. MiB for an array with shape (1780980, 78) and data type float32

In [27]:
counter = Counter(y_under)
benign = counter[0]

target = {}

for cls, count in counter.items():
    if cls == 0:
        continue
    target[cls] = min(count * 5, int(benign * 0.2))

In [28]:
smote = SMOTE(
    sampling_strategy=target,
    k_neighbors=5,
    random_state=42
)

X_res, y_res = smote.fit_resample(X_under, y_under)

print("After SMOTE:", Counter(y_res))

MemoryError: Unable to allocate 530. MiB for an array with shape (1780980, 78) and data type float32

In [21]:
rf = RandomForestClassifier(
    n_estimators=200,
    class_weight="balanced", 
    n_jobs=-1,
    random_state=42
)

rf.fit(X_res, y_res)

MemoryError: Unable to allocate 102. MiB for an array with shape (13357350,) and data type int64

In [ ]:
y_pred = rf.predict(X_test_scaled)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("F1 Score (macro):", f1_score(y_test, y_pred, average="macro"))

print("\nClassification Report:\n", classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.8031900222074487
F1 Score (macro): 0.059390303651909136

Classification Report:
               precision    recall  f1-score   support

           0       0.80      1.00      0.89    454265
           1       0.00      0.00      0.00       391
           2       0.00      0.00      0.00     25605
           3       0.00      0.00      0.00      2059
           4       0.00      0.00      0.00     46025
           5       0.00      0.00      0.00      1100
           6       0.00      0.00      0.00      1159
           7       0.00      0.00      0.00      1587
           8       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         7
          10       0.00      0.00      0.00     31761
          11       0.00      0.00      0.00      1180
          12       0.00      0.00      0.00       301
          13       0.00      0.00      0.00         4
          14       0.00      0.00      0.00       130

    accuracy                           0

C:\Users\Shantanu Ojha\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Shantanu Ojha\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Shantanu Ojha\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f